In [ ]:
from fastai.vision import *
import matplotlib.pyplot as py
import cv2
from tqdm import tqdm
from keras.utils import to_categorical
from keras.optimizers import SGD,Adam
from keras.callbacks import LearningRateScheduler
from PIL import Image
from sklearn.metrics import  accuracy_score
from sklearn.model_selection import train_test_split  
import pandas as pd
from  keras.layers import MaxPool2D,Conv2D,Dense,Flatten,Dropout,Activation,LeakyReLU,MaxoutDense
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from skimage import io
import os
import pandas as pd
print(os.listdir('../input/aptos2019-blindness-detection'))

%matplotlib inline

<h1>Data Preprocessing

In [ ]:
ptrain=get_image_files('../input/aptos2019-blindness-detection/train_images')

In [ ]:
file1=get_files('../input/aptos2019-blindness-detection')

In [ ]:
ptrain

In [ ]:
file1

In [ ]:
train=pd.read_csv(file1[2])

In [ ]:
train.head()

In [ ]:
X=[ ]
Y=[ ]
s='../input/aptos2019-blindness-detection/train_images/{}.png'

In [ ]:
for i in tqdm(range(train.shape[0])):
  Y.append(to_categorical(train.iloc[i,1],num_classes=5),)
  img=cv2.resize(cv2.imread(s.format(train.iloc[i,0])),(512,512))[:,:,1]
  X.append(img)
  


In [ ]:
cv2.imshow('eye',X[0])

<h2>Histogram Equalization</h2>





In [ ]:
X_m=[]

In [ ]:
for i in range(len(X)):
 X_m.append(cv2.equalizeHist(X[i]))

<h1>Kappa Score </h1>

In [ ]:
#credit to dan-blanchard on github
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Calculates the quadratic weighted kappa
    quadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return 1.0 - numerator / denominator

<h1>Modelling</h1>

In [ ]:
def learn(epoch):
    if epoch<200:
        return 0.003
    else:
        return 0.0003

In [ ]:
learnc=LearningRateScheduler(learn)

In [ ]:
Xt=np.array(X_m)
Yt=np.array(Y)

In [ ]:
x_train,x_val,y_train,y_val=train_test_split(Xt,Yt,test_size=0.3)

In [ ]:
img_rows=512
img_cols=512
if K.image_data_format=='channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1,img_cols,img_rows)
    x_val = x_val.reshape(x_val.shape[0], 1,img_cols,img_rows)
    input_shape = (1,img_cols,img_rows)
else:
    x_train = x_train.reshape(x_train.shape[0],img_cols,img_rows,1)
    x_val = x_val.reshape(x_val.shape[0],img_cols,img_rows,1)
    input_shape = (img_cols,img_rows,1)

In [ ]:
x_train[0]

In [ ]:
datagen=ImageDataGenerator(rescale=1.0/255.0)
train_iter=datagen.flow(x_train,y_train,batch_size=32)
val_iter=datagen.flow(x_val,y_val,batch_size=32)

In [ ]:
train_iter.x.shape

<h1>Model 1</h1>


In [ ]:
ml=Sequential()

In [ ]:
ml.add(Conv2D(filters=16,kernel_size=(5,5),strides=2,activation=LeakyReLU(0.01),input_shape=(512,512,1),padding='same'))
ml.add(Conv2D(filters=16,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(MaxPool2D(pool_size=(2,2),padding='same'))
ml.add(Dropout(0.1))
ml.add(Conv2D(filters=32,kernel_size=(3,3),strides=2,activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=32,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(MaxPool2D(pool_size=(2,2),padding='same'))
ml.add(Dropout(0.2))
ml.add(Conv2D(filters=48,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=48,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=48,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(MaxPool2D(pool_size=(2,2),padding='same'))
ml.add(Dropout(0.3))
ml.add(Conv2D(filters=64,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=64,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=64,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(MaxPool2D(pool_size=(2,2),padding='same'))
ml.add(Dropout(0.4))
ml.add(Conv2D(filters=128,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=128,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=128,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(MaxPool2D(pool_size=(2,2),padding='same'))
ml.add(Dropout(0.5))
ml.add(Flatten())
ml.add(Dense(units=400,activation=LeakyReLU(0.01)))
ml.add(MaxoutDense(200))
ml.add(Dropout(0.5))
ml.add(Dense(units=400,activation=LeakyReLU(0.01)))
ml.add(MaxoutDense(200))
ml.add(Dense(units=5,activation='softmax'))
ml.compile(loss='categorical_crossentropy',optimizer=SGD(momentum=0.9,lr=0.003,nesterov=True),metrics=['accuracy'])

In [ ]:
ml.fit_generator(train_iter,epochs=250,steps_per_epoch=len(train_iter),callbacks=[learnc])

In [ ]:
y_pred=ml.predict_classes(val_iter.x)

In [ ]:
y_val=np.argmax(val_iter.y,axis=1)

In [ ]:
print("kappa score is {}".format(quadratic_weighted_kappa(y_val,y_pred,min_rating=0,max_rating=4)))

<h1>Model 2</h1>

In [ ]:
ml=Sequential()

In [ ]:
ml.add(Conv2D(filters=16,kernel_size=(5,5),strides=2,activation=LeakyReLU(0.01),input_shape=(512,512,1),padding='same'))
ml.add(Conv2D(filters=16,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(MaxPool2D(pool_size=(2,2),padding='same'))
ml.add(Dropout(0.1))
ml.add(Conv2D(filters=32,kernel_size=(3,3),strides=2,activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=32,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(MaxPool2D(pool_size=(2,2),padding='same'))
ml.add(Dropout(0.2))
ml.add(Conv2D(filters=64,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=64,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=64,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(MaxPool2D(pool_size=(2,2),padding='same'))
ml.add(Dropout(0.3))
ml.add(Conv2D(filters=128,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=128,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=128,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(MaxPool2D(pool_size=(2,2),padding='same'))
ml.add(Dropout(0.4))
ml.add(Conv2D(filters=256,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=256,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=256,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(MaxPool2D(pool_size=(2,2),padding='same'))
ml.add(Dropout(0.5))
ml.add(Flatten())
ml.add(Dense(units=256,activation=LeakyReLU(0.01)))
ml.add(MaxoutDense(128))
ml.add(Dropout(0.5))
ml.add(Dense(units=256,activation=LeakyReLU(0.01)))
ml.add(MaxoutDense(128))
ml.add(Dense(units=5,activation='softmax'))
ml.compile(loss='categorical_crossentropy',optimizer=SGD(momentum=0.9,lr=0.003,nesterov=True))

In [ ]:
ml.fit_generator(train_iter,epochs=250,steps_per_epoch=len(train_iter),callbacks=[learnc])

In [ ]:
y_pred=ml.predict_classes(val_iter.x)
y_val=np.argmax(val_iter.y,axis=1)
print("kappa score is {}".format(quadratic_weighted_kappa(y_val,y_pred,min_rating=0,max_rating=4)))

<h1>Model 3</h1>

In [ ]:
ml=Sequential()

In [ ]:
ml.add(Conv2D(filters=16,kernel_size=(5,5),strides=2,activation=LeakyReLU(0.01),input_shape=(512,512,1),padding='same'))
ml.add(Conv2D(filters=16,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(MaxPool2D(pool_size=(2,2),padding='same'))
ml.add(Dropout(0.1))
ml.add(Conv2D(filters=32,kernel_size=(3,3),strides=2,activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=32,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(MaxPool2D(pool_size=(2,2),padding='same'))
ml.add(Dropout(0.2))
ml.add(Conv2D(filters=64,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=64,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=64,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(MaxPool2D(pool_size=(2,2),padding='same'))
ml.add(Dropout(0.3))
ml.add(Conv2D(filters=96,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=96,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=96,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(MaxPool2D(pool_size=(2,2),padding='same'))
ml.add(Dropout(0.4))
ml.add(Conv2D(filters=128,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=128,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(Conv2D(filters=128,kernel_size=(1,1),activation=LeakyReLU(0.01),padding='same'))
ml.add(MaxPool2D(pool_size=(2,2),padding='same'))
ml.add(Dropout(0.5))
ml.add(Flatten())
ml.add(Dense(units=256,activation=LeakyReLU(0.01)))
ml.add(MaxoutDense(128))
ml.add(Dropout(0.5))
ml.add(Dense(units=256,activation=LeakyReLU(0.01)))
ml.add(MaxoutDense(128))
ml.add(Dense(units=5,activation='softmax'))
ml.compile(loss='categorical_crossentropy',optimizer='Adam')

In [ ]:
ml.fit_generator(train_iter,epochs=250,steps_per_epoch=len(train_iter),callbacks=[learnc])

In [ ]:
y_pred=ml.predict_classes(train_iter.x)
y_val=np.argmax(train_iter.y,axis=1)
print("kappa score is {}".format(accuracy_score(y_val,y_pred)))